# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_Resp.db')
df = pd.read_sql_table('message', con=engine)
X = df['message']
Y = df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [3]:
URL_REGEX = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text: str) -> list:
    """
    Tokenizes the provided text. URLs in the text are replaced with "urlplaceholder".
    The text is then tokenized and lemmatized.

    Parameters:
    - text (str): The input text to be tokenized.

    Returns:
    - list: The cleaned and tokenized text.
    """
    # Detect and replace URLs
    text = re.sub(URL_REGEX, "urlplaceholder", text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize the tokens
    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)
category_name = Y_test.columns.tolist()
Y_pred_dtf = pd.DataFrame(Y_pred, columns = category_name)

def display_metrics(y_true, y_pred, category_names):
    """
    Display accuracy, precision, recall, and F1-score for each category.

    Parameters:
    - y_true: True labels
    - y_pred: Predicted labels
    - category_names: List of category names
    """
    for i, category in enumerate(category_names):
        acc = accuracy_score(y_true.iloc[:, i], y_pred.iloc[:, i])
        prec = precision_score(y_true.iloc[:, i], y_pred.iloc[:, i], average='micro')
        recal = recall_score(y_true.iloc[:, i], y_pred.iloc[:, i], average='micro')
        f1 = f1_score(y_true.iloc[:, i], y_pred.iloc[:, i], average='micro')
        
        print(f"{category} - Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {recal:.4f} | F1-Score: {f1:.4f}\n{'-'*40}")

display_metrics(Y_test, Y_pred_dtf, category_name)



related - Accuracy: 0.8001 | Precision: 0.8001 | Recall: 0.8001 | F1-Score: 0.8001
----------------------------------------
request - Accuracy: 0.8727 | Precision: 0.8727 | Recall: 0.8727 | F1-Score: 0.8727
----------------------------------------
offer - Accuracy: 0.9953 | Precision: 0.9953 | Recall: 0.9953 | F1-Score: 0.9953
----------------------------------------
aid_related - Accuracy: 0.7260 | Precision: 0.7260 | Recall: 0.7260 | F1-Score: 0.7260
----------------------------------------
medical_help - Accuracy: 0.9236 | Precision: 0.9236 | Recall: 0.9236 | F1-Score: 0.9236
----------------------------------------
medical_products - Accuracy: 0.9533 | Precision: 0.9533 | Recall: 0.9533 | F1-Score: 0.9533
----------------------------------------
search_and_rescue - Accuracy: 0.9730 | Precision: 0.9730 | Recall: 0.9730 | F1-Score: 0.9730
----------------------------------------
security - Accuracy: 0.9812 | Precision: 0.9812 | Recall: 0.9812 | F1-Score: 0.9812
----------------------

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f37c2e9a0d0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [8]:
# GridSearchCV
parameters = {'clf__estimator__max_depth': [5, 70, None],
              'clf__estimator__min_samples_leaf':[1, 7, 10]}
cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
cv.fit(X_train, Y_train) 

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [5, 70, None], 'clf__estimator__min_samples_leaf': [1, 7, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
# predict and create a dataframe
Y_pred = cv.predict(X_test)
Y_pred_df = pd.DataFrame(Y_pred, columns = category_name)
display_metrics(Y_test, Y_pred_df, category_name)

related - Accuracy: 0.8036 | Precision: 0.8036 | Recall: 0.8036 | F1-Score: 0.8036
----------------------------------------
request - Accuracy: 0.8808 | Precision: 0.8808 | Recall: 0.8808 | F1-Score: 0.8808
----------------------------------------
offer - Accuracy: 0.9953 | Precision: 0.9953 | Recall: 0.9953 | F1-Score: 0.9953
----------------------------------------
aid_related - Accuracy: 0.7302 | Precision: 0.7302 | Recall: 0.7302 | F1-Score: 0.7302
----------------------------------------
medical_help - Accuracy: 0.9272 | Precision: 0.9272 | Recall: 0.9272 | F1-Score: 0.9272
----------------------------------------
medical_products - Accuracy: 0.9532 | Precision: 0.9532 | Recall: 0.9532 | F1-Score: 0.9532
----------------------------------------
search_and_rescue - Accuracy: 0.9719 | Precision: 0.9719 | Recall: 0.9719 | F1-Score: 0.9719
----------------------------------------
security - Accuracy: 0.9809 | Precision: 0.9809 | Recall: 0.9809 | F1-Score: 0.9809
----------------------

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# Define the pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# Train the pipeline
pipeline.fit(X_train, Y_train)

# Predict using the pipeline
Y_pred = pipeline.predict(X_test)

# Save the predicted values into a DataFrame
Y_pred_df = pd.DataFrame(Y_pred, columns=category_name)

display_metrics(Y_test, Y_pred_df, category_name)

related - Accuracy: 0.7438 | Precision: 0.7438 | Recall: 0.7438 | F1-Score: 0.7438
----------------------------------------
request - Accuracy: 0.8488 | Precision: 0.8488 | Recall: 0.8488 | F1-Score: 0.8488
----------------------------------------
offer - Accuracy: 0.9919 | Precision: 0.9919 | Recall: 0.9919 | F1-Score: 0.9919
----------------------------------------
aid_related - Accuracy: 0.6977 | Precision: 0.6977 | Recall: 0.6977 | F1-Score: 0.6977
----------------------------------------
medical_help - Accuracy: 0.8943 | Precision: 0.8943 | Recall: 0.8943 | F1-Score: 0.8943
----------------------------------------
medical_products - Accuracy: 0.9385 | Precision: 0.9385 | Recall: 0.9385 | F1-Score: 0.9385
----------------------------------------
search_and_rescue - Accuracy: 0.9634 | Precision: 0.9634 | Recall: 0.9634 | F1-Score: 0.9634
----------------------------------------
security - Accuracy: 0.9675 | Precision: 0.9675 | Recall: 0.9675 | F1-Score: 0.9675
----------------------

### 9. Export your model as a pickle file

In [14]:
pickle.dump(cv, open('model_class.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.